# Bagged Decision Trees

是一種集成學習方法，它結合了決策樹和重複抽樣（bootstrap aggregating）的概念。Bagging 通常用於改善單個決策樹的穩定性和泛化能力。以下是 Bagged Decision Trees 的基本概念和運作方式：

1. 重複抽樣（Bootstrap Aggregating）：
   
   Bagged Decision Trees 使用重複抽樣方法，也稱為 bootstrap，從原始訓練數據集中隨機選取多個子樣本，每個子樣本的大小通常與原始數據集相同。這樣可以創建多個不同的子數據集。

2. 多個決策樹：
   
   對於每個子樣本，建立一個獨立的決策樹模型。由於每個決策樹在不同的子樣本上訓練，它們之間有差異，因此每個決策樹都可以捕捉到不同的特徵和模式。

3. 結合預測：
   
   在預測階段，對於分類問題，Bagged Decision Trees 會將每個決策樹的預測結果進行投票，選擇得票最多的類別作為最終預測結果。對於回歸問題，則是計算所有決策樹的預測值的平均值。

## 和 Random Frests 的差異

1. 特徵選擇：

    * Bagged Decision Trees：每個決策樹在訓練時，使用所有特徵來進行切割。這意味著每個決策樹可能會選擇相同的特徵來進行切割，導致模型間的相似性較高。
    * Random Forests：每個決策樹在切割特徵時，只會從隨機選定的特徵子集中選擇。這種隨機性使得每個決策樹更加多樣化，提高了整體模型的性能。
---
2. 決策樹數量：

    * Bagged Decision Trees：通常會使用較多的決策樹，因為每個決策樹的多樣性較低，需要更多的模型來進行平均。
    * Random Forests：由於決策樹的多樣性較高，通常可以使用較少的決策樹來達到類似的性能。
---
3. 預測方式：

    * Bagged Decision Trees：對於分類問題，通常採用多數決的方式進行預測。對於回歸問題，則取決於平均預測值。
    * Random Forests：同樣的預測方式，但通常由於每個決策樹使用隨機特徵，它們的預測結果之間的相關性較低。
---
4. 效能：

    * Random Forests：由於使用了特徵隨機性，通常比單純的 
    * Bagged Decision Trees 有更好的性能。它更容易抵抗過擬合，並且能夠捕捉更複雜的模式。

## 優缺點

### 優點

* 減少過擬合：由於每個決策樹都在不同的子樣本上訓練，它們的泛化能力更好，有助於減少過擬合的風險。
* 提高穩定性：Bagging 可以降低單個模型的方差，提高整體模型的穩定性。
* 容易平行化：每個決策樹的訓練是獨立的，因此容易在多核心或分布式環境中平行運行。

### 缺點

* 由於模型之間的相似性，它可能無法捕捉到複雜的非線性模式。這在某些情況下可能導致性能不佳。

## python

```python
from sklearn.ensemble import BaggingClassifier
...
bagged_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)
...
```

* estimator: 基模型，即要進行 Bagging 的基本學習器，預設為 DecisionTreeClassifier()。

* n_estimators: 基模型的數量，即要生成多少個基模型進行集成。

* max_samples: 每個基模型訓練時的樣本數，可以是絕對數量，也可以是小數（例如 0.5 表示使用 50% 的樣本）。

* max_features: 每個基模型訓練時的特徵數，可以是絕對數量，也可以是小數。

* bootstrap: 是否使用重複抽樣，預設為 True。

* bootstrap_features: 是否對特徵進行重複抽樣，預設為 False。

* random_state: 隨機種子，用於確保結果的可重現性。

* n_jobs: 同時運行的作業數，預設為 1。如果設置為 -1，則使用所有的可用 CPU。

In [2]:
from sklearn.datasets import load_iris
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 載入 Iris 資料集
iris = load_iris()
X = iris.data
y = iris.target

# 分割資料集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 創建單一 Decision Tree 分類器作為基模型
base_model = DecisionTreeClassifier(random_state=42)

# 創建 Bagging 分類模型，使用 10 個基模型
bagged_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)

# 訓練模型
bagged_model.fit(X_train, y_train)

# 預測測試集
y_pred = bagged_model.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print("Bagged Decision Trees 模型的準確率:", accuracy)


Bagged Decision Trees 模型的準確率: 1.0


## python

手動寫 bagged

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 載入 Iris 資料集
iris = load_iris()
X = iris.data
y = iris.target

# 分割資料集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定義 Bagging 集成的基模型數量
n_estimators = 10

# 創建空的預測結果列表
y_pred_bagged = []

# 創建 Bagging 集成
for _ in range(n_estimators):
    # 隨機取樣訓練集，可以使用 replacement=True 表示有放回取樣
    indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_sampled = X_train[indices]
    y_sampled = y_train[indices]
    
    # 創建單一 Decision Tree 分類器作為基模型
    base_model = DecisionTreeClassifier(random_state=42)
    
    # 訓練基模型
    base_model.fit(X_sampled, y_sampled)
    
    # 預測測試集並將結果添加到預測結果列表中
    y_pred = base_model.predict(X_test)
    y_pred_bagged.append(y_pred)

# 對預測結果進行投票，取最多票的類別為最終預測結果
y_pred_bagged = np.array(y_pred_bagged)
final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=y_pred_bagged)

# 計算準確率
accuracy = accuracy_score(y_test, final_predictions)
print("手動實現 Bagged Decision Trees 模型的準確率:", accuracy)


手動實現 Bagged Decision Trees 模型的準確率: 1.0
